In [22]:
###################################注意事项######################################
# cv2读取图片的宽高和其他函数要求的顺序可能是不同的
##################################################################################

# Standard imports
import cv2
import numpy as np 

# Read images
src = cv2.imread("images/DSC_0182_780.jpg")
dst = cv2.imread("images/background.jpg")

# Create a rough mask around the airplane.
src_mask = np.zeros(src.shape, src.dtype)

# Resize
# dst = cv2.resize(dst, (500, 300), interpolation = cv2.INTER_CUBIC)

poly = np.array([ 
        [
          951.9890109890109,
          512.0879120879121
        ],
        [
          934.4065934065934,
          486.8131868131868
        ],
        [
          920.1208791208792,
          476.9230769230769
        ],
        [
          901.4395604395604,
          478.021978021978
        ],
        [
          882.7582417582416,
          473.6263736263736
        ],
        [
          876.164835164835,
          459.34065934065933
        ],
        [
          886.0549450549449,
          431.86813186813185
        ],
        [
          895.9450549450548,
          415.38461538461536
        ],
        [
          913.5274725274726,
          394.5054945054945
        ],
        [
          942.098901098901,
          357.1428571428571
        ],
        [
          960.7802197802198,
          348.35164835164835
        ],
        [
          981.6593406593406,
          348.35164835164835
        ],
        [
          997.0439560439561,
          342.85714285714283
        ],
        [
          1027.8131868131868,
          335.16483516483515
        ],
        [
          1041.0,
          334.0659340659341
        ],
        [
          1057.4835164835165,
          346.15384615384613
        ],
        [
          1069.5714285714284,
          365.9340659340659
        ],
        [
          1062.9780219780218,
          397.8021978021978
        ],
        [
          1049.7912087912086,
          409.8901098901099
        ],
        [
          1025.6153846153845,
          428.57142857142856
        ],
        [
          995.9450549450548,
          427.4725274725275
        ],
        [
          967.3736263736264,
          435.16483516483515
        ],
        [
          957.4835164835165,
          447.2527472527472
        ],
        [
          953.087912087912,
          460.4395604395604
        ],
        [
          947.5934065934066,
          474.7252747252747
        ],
        [
          947.5934065934066,
          480.2197802197802
        ],
        [
          954.1868131868132,
          489.010989010989
        ] ],  np.int32)
    
cv2.fillPoly(src_mask, [poly], (255, 255, 255)) #原图的mask
src_mask.shape 

(1080, 1920, 3)

In [23]:
y_max, y_min, x_max, x_min = np.max(poly[:, 0]), np.min(poly[:, 0]), np.max(poly[:, 1]), np.min(poly[:, 1]) # mask最小方框的坐标
width = np.max(poly[:, 0]) - np.min(poly[:, 0]) # 最小方框的宽
height =  np.max(poly[:, 1]) - np.min(poly[:, 1]) # 最小方框的高 

# print(int(x_min), int(x_max), int(y_min), int(y_max))
# 这是 CENTER 所在的地方
center = (233, 133)  # center 是(y, x) cv2.imread 是(x, y), poly 是(y,x) x 是垂直方向， y 是水平方向

# src 中的roi
src_roi = src[int(x_min): int(x_max), int(y_min): int(y_max)]

# dst 中的roi
dst_roi = dst[int(center[1] - height / 2): int(center[1] + height / 2), int(center[0] - width / 2): int(center[0] + width / 2)]

# src_mask 中的roi
mask_roi = src_mask[int(x_min): int(x_max), int(y_min): int(y_max)]

mask_roi.shape  # 三通道的

(178, 193, 3)

In [24]:
# 按位操作
img2gray = cv2.cvtColor(mask_roi,cv2.COLOR_BGR2GRAY)      # 将图片灰度化
ret, mask = cv2.threshold(img2gray, 175, 255, cv2.THRESH_BINARY)#ret是阈值（175）mask是二值化图像， 虽然已经是二值化的图片了，但是没有这步效果很差，不知道为啥 
mask_inv = cv2.bitwise_not(mask)#获取把logo的区域取反 按位运算


src_smoke_only  =  cv2.bitwise_and(src_roi,src_roi,mask = mask) # 截取烟雾，非截取部分取值为0
dst_smoke_only  =  cv2.bitwise_and(src_roi,src_roi,mask = mask) # 截取烟雾在背景区域的部分，非截取部分取值为0
dst_bg_only  =  cv2.bitwise_and(dst_roi,dst_roi,mask = mask_inv) # 截取背景
smoke = cv2.addWeighted(src_smoke_only, 0.5, dst_smoke_only, 0.5, 0)
add = cv2.add(smoke,dst_bg_only)#相加即可

In [25]:
cv2.imwrite("images/add_samll.jpg", add)

True

In [26]:
dst[int(center[1] - height / 2): int(center[1] + height / 2), int(center[0] - width / 2): int(center[0] + width / 2)] = add
cv2.imwrite("images/dst_add_small.jpg", dst)

True

In [27]:
seamless = cv2.imread('./images/mixed.jpg')
smoke_new = cv2.addWeighted(dst, 0.5, seamless, 0.5, 0)
cv2.imwrite("images/double_add.jpg", smoke_new)

error: OpenCV(4.0.0) /io/opencv/modules/core/src/arithm.cpp:663: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'arithm_op'


In [1]:
###################################注意事项######################################
# cv2读取图片的宽高和其他函数要求的顺序可能是不同的
##################################################################################

# Standard imports
import cv2
import numpy as np 

# Read images
src = cv2.imread("images/DSC_0182_780.jpg")
dst = cv2.imread("images/background.jpg")

# Create a rough mask around the airplane.
src_mask = np.zeros(src.shape, src.dtype)

# Resize
# dst = cv2.resize(dst, (500, 300), interpolation = cv2.INTER_CUBIC)

poly = np.array([ 
        [
          951.9890109890109,
          512.0879120879121
        ],
        [
          934.4065934065934,
          486.8131868131868
        ],
        [
          920.1208791208792,
          476.9230769230769
        ],
        [
          901.4395604395604,
          478.021978021978
        ],
        [
          882.7582417582416,
          473.6263736263736
        ],
        [
          876.164835164835,
          459.34065934065933
        ],
        [
          886.0549450549449,
          431.86813186813185
        ],
        [
          895.9450549450548,
          415.38461538461536
        ],
        [
          913.5274725274726,
          394.5054945054945
        ],
        [
          942.098901098901,
          357.1428571428571
        ],
        [
          960.7802197802198,
          348.35164835164835
        ],
        [
          981.6593406593406,
          348.35164835164835
        ],
        [
          997.0439560439561,
          342.85714285714283
        ],
        [
          1027.8131868131868,
          335.16483516483515
        ],
        [
          1041.0,
          334.0659340659341
        ],
        [
          1057.4835164835165,
          346.15384615384613
        ],
        [
          1069.5714285714284,
          365.9340659340659
        ],
        [
          1062.9780219780218,
          397.8021978021978
        ],
        [
          1049.7912087912086,
          409.8901098901099
        ],
        [
          1025.6153846153845,
          428.57142857142856
        ],
        [
          995.9450549450548,
          427.4725274725275
        ],
        [
          967.3736263736264,
          435.16483516483515
        ],
        [
          957.4835164835165,
          447.2527472527472
        ],
        [
          953.087912087912,
          460.4395604395604
        ],
        [
          947.5934065934066,
          474.7252747252747
        ],
        [
          947.5934065934066,
          480.2197802197802
        ],
        [
          954.1868131868132,
          489.010989010989
        ] ],  np.int32)
    
## 生成mask
cv2.fillPoly(src_mask, [poly], (255, 255, 255)) #原图的mask
src_mask.shape 

(1080, 1920, 3)

In [2]:
##  获取包裹mask最小方框的信息

y_max, y_min, x_max, x_min = np.max(poly[:, 0]), np.min(poly[:, 0]), np.max(poly[:, 1]), np.min(poly[:, 1]) # mask最小方框的坐标
width = np.max(poly[:, 0]) - np.min(poly[:, 0]) # 最小方框的宽
height =  np.max(poly[:, 1]) - np.min(poly[:, 1]) # 最小方框的高 

In [4]:
## 获取src,dst和mask中的对应的方框

src_roi = src[int(x_min): int(x_max), int(y_min): int(y_max)] # src 中的roi
mask_roi = src_mask[int(x_min): int(x_max), int(y_min): int(y_max)] # src_mask 中的roi

# 这是 CENTER 所在的地方
center = (233, 133)  # center 是(y, x) cv2.imread 是(x, y), poly 是(y,x) x 是垂直方向， y 是水平方向
dst_roi = dst[int(center[1] - height / 2): int(center[1] + height / 2), int(center[0] - width / 2): int(center[0] + width / 2)] # dst 中的roi

In [8]:
## 按位操作，获取
img2gray = cv2.cvtColor(mask_roi,cv2.COLOR_BGR2GRAY)      # 将图片灰度化,变成单通道
ret, mask = cv2.threshold(img2gray, 175, 255, cv2.THRESH_BINARY)#ret是阈值（175）mask是二值化图像， 虽然已经是二值化的图片了，但是没有这步效果很差，不知道为啥 
mask_inv = cv2.bitwise_not(mask)#获取把logo的区域取反按位运算


src_smoke_only  =  cv2.bitwise_and(src_roi,src_roi,mask = mask) # 截取烟雾，非截取部分取值为0
dst_smoke_only  =  cv2.bitwise_and(dst_roi,dst_roi,mask = mask) # 截取烟雾在背景区域的部分，非截取部分取值为0
dst_bg_only  =  cv2.bitwise_and(dst_roi,dst_roi,mask = mask_inv) # 截取背景
new_smoke= cv2.addWeighted(src_smoke_only, 0.5, dst_smoke_only, 0.5, 0)
add = cv2.add(new_smoke, dst_bg_only)#相加即可

In [ ]:
# 使用cv2.imshow()的时候，如果图片太大，会显示不全并且无法调整，因此在cv2.imshow()的前面加上这样的一个语句：cv2.namedWindow('image', 0)，得到的图像框就可以自己调整大小，按住四个角会出来小箭头可以拉伸调整。
cv2.namedWindow('image')